In [107]:
#  checking data types and any potential issues
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# Load data
df = pd.read_csv(r"C:\Users\LADY FEJ\healthypact_dataset.csv")

# Display data information
print("Dataset Info:")
print(df.info())
print("\nMissing Values:")
print(df.isnull().sum())
print("\nData Types:")
print(df.dtypes)
print("\nSample Data:")
print(df.head())



Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 730 entries, 0 to 729
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   date                    730 non-null    object 
 1   store_id                730 non-null    int64  
 2   product_id              730 non-null    int64  
 3   demand                  730 non-null    int64  
 4   price                   730 non-null    float64
 5   promotion               730 non-null    int64  
 6   economic_index          730 non-null    float64
 7   competitor_price_index  730 non-null    float64
dtypes: float64(3), int64(4), object(1)
memory usage: 45.8+ KB
None

Missing Values:
date                      0
store_id                  0
product_id                0
demand                    0
price                     0
promotion                 0
economic_index            0
competitor_price_index    0
dtype: int64

Data Types:
date             

In [108]:
# Importing libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
import joblib


In [109]:

df = pd.read_csv(r"C:\Users\LADY FEJ\healthypact_dataset.csv")

In [110]:
# Create time-based features
df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y')
df['month'] = df['date'].dt.month
df['day_of_week'] = df['date'].dt.dayofweek
df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)

# Create seasonal features
df['month_sin'] = np.sin(2 * np.pi * df['month']/12)
df['month_cos'] = np.cos(2 * np.pi * df['month']/12)

# Create demand features with proper grouping
grouped = df.groupby(['store_id', 'product_id'])
df['demand_lag_7'] = grouped['demand'].transform(lambda x: x.shift(7))
df['demand_lag_14'] = grouped['demand'].transform(lambda x: x.shift(14))
df['rolling_mean_7'] = grouped['demand'].transform(lambda x: x.rolling(window=7).mean())

# Remove rows with NaN values
df_clean = df.dropna()

# Select final feature set
features = ['month_sin', 'month_cos', 'is_weekend', 'price', 
            'promotion', 'economic_index', 'competitor_price_index',
            'demand_lag_7', 'demand_lag_14', 'rolling_mean_7']

In [111]:
# Prepare features and target
X = df_clean[features].values
y = df_clean['demand'].values


In [135]:

df = pd.read_csv(r"C:\Users\LADY FEJ\healthypact_dataset.csv")
print("Initial sample count:", len(df))
df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y')
df['month'] = df['date'].dt.month
df['day_of_week'] = df['date'].dt.dayofweek
df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)
df['month_sin'] = np.sin(2 * np.pi * df['month']/12)
df['month_cos'] = np.cos(2 * np.pi * df['month']/12)
grouped = df.groupby(['store_id', 'product_id'])['demand']
df['demand_lag_7'] = grouped.transform(lambda x: x.shift(7).fillna(x.mean()))
df['demand_lag_14'] = grouped.transform(lambda x: x.shift(14).fillna(x.mean()))
df['rolling_mean_7'] = grouped.transform(lambda x: x.rolling(window=7, min_periods=1).mean())

print("Sample count after feature creation:", len(df))
features = ['month_sin', 'month_cos', 'is_weekend', 'price', 
            'promotion', 'economic_index', 'competitor_price_index',
            'demand_lag_7', 'demand_lag_14', 'rolling_mean_7']

# Verify feature data
X = df[features].values
print("Feature matrix shape:", X.shape)

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
print("Scaled feature matrix shape:", X_scaled.shape)


Initial sample count: 730
Sample count after feature creation: 730
Feature matrix shape: (730, 10)
Scaled feature matrix shape: (730, 10)


In [113]:
# Create time features
df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y')
df['month'] = df['date'].dt.month
df['day_of_week'] = df['date'].dt.dayofweek
df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)

# Create seasonal features
df['month_sin'] = np.sin(2 * np.pi * df['month']/12)
df['month_cos'] = np.cos(2 * np.pi * df['month']/12)

# Create demand features
grouped = df.groupby(['store_id', 'product_id'])['demand']
df['demand_lag_7'] = grouped.transform(lambda x: x.shift(7).fillna(x.mean()))
df['demand_lag_14'] = grouped.transform(lambda x: x.shift(14).fillna(x.mean()))
df['rolling_mean_7'] = grouped.transform(lambda x: x.rolling(window=7, min_periods=1).mean())

# Select features
features = ['month_sin', 'month_cos', 'is_weekend', 'price', 
            'promotion', 'economic_index', 'competitor_price_index',
            'demand_lag_7', 'demand_lag_14', 'rolling_mean_7']

# Prepare features and target
X = df[features].values
y = df['demand'].values  

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

print("Training set shape:", X_train.shape)
print("Test set shape:", X_test.shape)


Training set shape: (584, 10)
Test set shape: (146, 10)


In [114]:
# Split the scaled data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [115]:
# Train the model with the prepared data
model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
model.fit(X_train, y_train)

# Evaluate model performance
predictions = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, predictions))
r2 = r2_score(y_test, predictions)

print("\nModel Performance Metrics:")
print(f"RMSE: {rmse:.2f}")
print(f"R2 Score: {r2:.2f}")




Model Performance Metrics:
RMSE: 202.36
R2 Score: 0.79


In [133]:
# Price optimization 
def optimize_price(row_features, price_range=np.arange(10, 100, 1)):
    best_price = 0
    max_revenue = 0
    price_idx = features.index('price')
    
    for price in price_range:
        row_copy = row_features.copy()
        row_copy[price_idx] = price
        scaled_features = scaler.transform([row_copy])
        predicted_demand = model.predict(scaled_features)[0]
        revenue = predicted_demand * price
        
        if revenue > max_revenue:
            max_revenue = revenue
            best_price = price
    
    return best_price, max_revenue


In [131]:
# Sample optimization
sample_features = X[0]
optimal_price, expected_revenue = optimize_price(sample_features)

print("\nPrice Optimization Results:")
print(f"Optimal Price: ${optimal_price:.2f}")
print(f"Expected Revenue: ${expected_revenue:.2f}")



Price Optimization Results:
Optimal Price: $97.00
Expected Revenue: $139964.85


In [118]:
# Save model and scaler
joblib.dump(model, 'demand_forecaster_model.joblib')
joblib.dump(scaler, 'feature_scaler.joblib')
print("\nModel and scaler saved successfully")


Model and scaler saved successfully


In [141]:
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
import pandas as pd

# 1. Seasonal Decomposition
plt.figure(figsize=(15, 10))
# Convert date to index for decomposition
df_temp = df.set_index('date')
decomposition = seasonal_decompose(df_temp['demand'], period=30)

plt.subplot(411)
plt.plot(decomposition.observed)
plt.title('Observed Demand')
plt.subplot(412)
plt.plot(decomposition.trend)
plt.title('Demand Trend')
plt.subplot(413)
plt.plot(decomposition.seasonal)
plt.title('Seasonal Pattern')
plt.subplot(414)
plt.plot(decomposition.resid)
plt.title('Residuals')
plt.tight_layout()
plt.savefig('seasonal_decomposition.png', dpi=300)
plt.close()

# 2. Rolling Demand Metrics
plt.figure(figsize=(12, 6))
rolling_7 = df.groupby('date')['demand'].mean().rolling(window=7).mean()
rolling_30 = df.groupby('date')['demand'].mean().rolling(window=30).mean()

plt.plot(rolling_7.index, rolling_7, label='7-Day Rolling Average', color='blue')
plt.plot(rolling_30.index, rolling_30, label='30-Day Rolling Average', color='red')
plt.title('Rolling Demand Metrics', fontsize=14)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Demand', fontsize=12)
plt.legend()
plt.grid(True)
plt.savefig('rolling_demand.png', dpi=300)
plt.close()

# 3. Economic Indicators Impact
plt.figure(figsize=(12, 6))
fig, ax1 = plt.subplots()

ax1.set_xlabel('Date')
ax1.set_ylabel('Demand', color='tab:blue')
ax1.plot(df['date'], df['demand'], color='tab:blue')
ax1.tick_params(axis='y', labelcolor='tab:blue')

ax2 = ax1.twinx()
ax2.set_ylabel('Economic Index', color='tab:red')
ax2.plot(df['date'], df['economic_index'], color='tab:red')
ax2.tick_params(axis='y', labelcolor='tab:red')

plt.title('Demand vs Economic Indicators')
fig.tight_layout()
plt.savefig('economic_indicators.png', dpi=300)
plt.close()

# 4. Competitive Price Index Analysis
plt.figure(figsize=(12, 6))
plt.scatter(df['competitor_price_index'], df['demand'], alpha=0.5)
plt.title('Competitive Price Index vs Demand', fontsize=14)
plt.xlabel('Competitor Price Index', fontsize=12)
plt.ylabel('Demand', fontsize=12)
plt.grid(True)
plt.savefig('competitive_price_index.png', dpi=300)
plt.close()

print("Visualization files created successfully:")
print("1. seasonal_decomposition.png")
print("2. rolling_demand.png")
print("3. economic_indicators.png")
print("4. competitive_price_index.png")

# Display key insights
print("\nKey Insights:")
print(f"Seasonal Peak Amplitude: {decomposition.seasonal.max():.2f}")
print(f"30-Day Rolling Average Range: {rolling_30.min():.2f} - {rolling_30.max():.2f}")
print(f"Economic Index Correlation: {df['demand'].corr(df['economic_index']):.2f}")
print(f"Competitive Index Impact: {df['demand'].corr(df['competitor_price_index']):.2f}")

Visualization files created successfully:
1. seasonal_decomposition.png
2. rolling_demand.png
3. economic_indicators.png
4. competitive_price_index.png

Key Insights:
Seasonal Peak Amplitude: 233.67
30-Day Rolling Average Range: 882.10 - 1789.70
Economic Index Correlation: 0.32
Competitive Index Impact: -0.28


<Figure size 960x480 with 0 Axes>